In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from keras.preprocessing.image import load_img

df_coord = pd.read_csv("./data/coordinates.csv", names=["COD_SETOR","ID", "LAT", "LNG"])
df_geo = gpd.read_file("./data/income_map/income_map.shp")
df_geo["COD_SETOR"] = pd.to_numeric(df_geo["COD_SETOR"],downcast="integer")

df_coord.set_index("COD_SETOR", inplace=True)
df_geo.set_index("COD_SETOR", inplace=True)
df_final = df_coord.join(
    df_geo["LOG_MEAN_I"],
    how="inner"
)
df_final

Using TensorFlow backend.


,ID,LAT,LNG,LOG_MEAN_I
COD_SETOR,,,,
355030801000001,0,-23.568380,-46.572104,7.034142
355030801000001,1,-23.566752,-46.570455,7.034142
355030801000001,2,-23.568870,-46.569899,7.034142
355030801000001,3,-23.567367,-46.572305,7.034142
355030801000001,4,-23.567225,-46.570765,7.034142
355030801000001,5,-23.567066,-46.572299,7.034142
355030801000001,6,-23.566675,-46.570208,7.034142
355030801000001,7,-23.568624,-46.571871,7.034142
355030801000001,8,-23.567973,-46.569667,7.034142


In [2]:
import os

success = []

for census_dir in os.listdir("./images"):
    fullpath = os.path.join("./images",census_dir)
    if len(os.listdir(fullpath)) == 10:
        success.append(int(census_dir))
        
len(success)

11881

In [3]:
def get_image(sector, iid):    
    fullpath = "./images/{sector:15d}/IMG_{sector:015d}_{iid:03d}.jpg".format(sector=int(sector), iid=int(iid))
    return load_img(fullpath, target_size=(244,244))

def get_image_batch(batch_ids):
    nrows = batch_ids.shape[0]
    features = np.zeros((nrows, 244, 244,3))
    
    for idx, row in enumerate(batch_ids.iterrows()):
        sector, data = row
        iid = data["ID"]
        try:
            features[idx,:,:,:] = get_image(sector, iid)
        except ValueError as e:
            print(sector, iid)
        
    labels = batch_ids["LOG_MEAN_I"].values
    return features, labels
        

def batch_generator(df, batch_size=10000, shuffle=True):
    #Shuffling the dataframe
    if shuffle:
        df_int = df.sample(frac=1)
    else:
        df_int = df
        
    batch = 0
    end = batch_size
    while end < df_int.shape[0]:
        start = batch*batch_size
        end = start + batch_size
        current = df_int[start:end]
        yield get_image_batch(current)
        batch = batch + 1

df = df_final[df_final.index.isin(success)]      
        
gen = batch_generator(df, batch_size=10000)
features, labels = next(gen)

In [4]:
from keras.models import Model
from keras.layers import Convolution2D, Dense, Dropout, MaxPooling2D, Input, Flatten, ZeroPadding2D

x_train = features[0:7000]
y_train = labels[0:7000]

x_test = features[7000:10000]
y_test = labels[7000:10000]

inputs = Input(shape=(244,244,3))

x = ZeroPadding2D((1,1))(inputs)
x = Convolution2D(64,(3,3), activation="relu")(x)
x = ZeroPadding2D((1,1))(inputs)
x = Convolution2D(64,(3,3), activation="relu")(x)
x = MaxPooling2D((2,2))(x)

#x = ZeroPadding2D((1,1))(inputs)
#x = Convolution2D(64,(3,3), activation="relu")(x)
#x = ZeroPadding2D((1,1))(inputs)
#x = Convolution2D(64,(3,3), activation="relu")(x)
#x = MaxPooling2D((2,2))(x)

#x = ZeroPadding2D((1,1))(inputs)
#x = Convolution2D(64,(3,3), activation="relu")(x)
#x = ZeroPadding2D((1,1))(inputs)
#x = Convolution2D(64,(3,3), activation="relu")(x)
#x = ZeroPadding2D((1,1))(inputs)
#x = Convolution2D(64,(3,3), activation="relu")(x)
#x = MaxPooling2D((2,2))(x)

x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1)(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])


model.fit(x=x_train, y=y_train, batch_size=4, epochs=10, validation_data=(x_test, y_test))

Train on 7000 samples, validate on 3000 samples
Epoch 1/10
7000/7000 [==============================] - 336s - loss: 589687.5007 - mean_absolute_error: 75.8595 - val_loss: 44.8684 - val_mean_absolute_error: 6.6111
Epoch 2/10
7000/7000 [==============================] - 332s - loss: 44.2650 - mean_absolute_error: 6.5666 - val_loss: 43.4964 - val_mean_absolute_error: 6.5089
Epoch 3/10
7000/7000 [==============================] - 334s - loss: 42.0811 - mean_absolute_error: 6.4013 - val_loss: 40.3270 - val_mean_absolute_error: 6.2662
Epoch 4/10
7000/7000 [==============================] - 336s - loss: 37.5035 - mean_absolute_error: 6.0371 - val_loss: 34.0655 - val_mean_absolute_error: 5.7559
Epoch 5/10
7000/7000 [==============================] - 335s - loss: 29.5300 - mean_absolute_error: 5.3386 - val_loss: 24.2643 - val_mean_absolute_error: 4.8473
Epoch 6/10
7000/7000 [==============================] - 335s - loss: 18.8027 - mean_absolute_error: 4.2091 - val_loss: 13.2480 - val_mean_abso

In [18]:
np.max(model.predict(x_train))

6.4425573

In [19]:
np.min(model.predict(x_train))

6.4425573